In [49]:
import pandas as pd
import numpy as np
from sleeper_wrapper import League

In [50]:
def map_rosterid_to_ownerid(rosters):
    """returns: dict {roster_id:[owner_id,pts]}"""
    result_dict = {}
    for roster in rosters:
        roster_id = roster["roster_id"]
        owner_id = roster["owner_id"]
        result_dict[roster_id] = owner_id

    return result_dict

In [51]:
def map_users_to_team_name(users):
    """returns dict {user_id:team_name}"""
    users_dict = {}

    # Maps the user_id to team name for easy lookup
    for user in users:
        users_dict[user["user_id"]] = user["display_name"]
    return users_dict

In [80]:
def get_weekly_data(rosters, matchups, users, week, current_period):
    """returns dict {matchup_id:[(team_name,score), (team_name, score)]}"""

    roster_id_dict = map_rosterid_to_ownerid(rosters)

    if len(matchups) == 0:
        return None

    # get the users to team name stats
    users_dict = map_users_to_team_name(users)

    # getting base scores
    scoreboards_dict = {}
    for team in matchups:
        # confirming matchups
        matchup_id = team["matchup_id"]
        current_roster_id = team["roster_id"]
        owner_id = roster_id_dict[current_roster_id]

        if owner_id is not None:
            user_name = users_dict[owner_id]
        else:
            user_name = "User name not available"
        # this is broken
        # team_score = self.get_team_score(team["starters"], score_type, week)
        # getting data of interest
        points_for = team["points"]
        if points_for is None:
            points_for = 0

        team_score_tuple = (owner_id, user_name, points_for)
        if matchup_id not in scoreboards_dict:
            scoreboards_dict[matchup_id] = [team_score_tuple]
        else:
            scoreboards_dict[matchup_id].append(team_score_tuple)

    # converting matchups to list for parsing
    data = []
    for _, value in scoreboards_dict.items():
        data.append(value[0] + value[1])
        data.append(value[1] + value[0])

    # generating team data
    final_team_data = []
    for team in data:
        team_data = {}
        team_data["user_id"] = team[0]
        team_data["user_name"] = team[1]
        team_data["week"] = week
        team_data["opponent_id"] = team[3]
        team_data["opponent_name"] = team[4]
        team_data["points_for"] = team[2]
        team_data["points_against"] = team[5]
        team_data["points_spread"] = round(team[2] - team[5], 2)
        if week < 15:
            team_data["playoffs"] = False
        else:
            team_data["playoffs"] = True

        final_team_data.append(team_data)

    df = pd.DataFrame(final_team_data)

    # setting wins, losses, and ties
    df["wins"] = np.where(df["points_for"] > df["points_against"], 1, 0)
    df["losses"] = np.where(df["points_for"] < df["points_against"], 1, 0)
    df["ties"] = np.where(df["points_for"] == df["points_against"], 1, 0)
    df["current_period"] = current_period

    return df

In [81]:
def get_roster_stats(rosters):
    # generating roster data
    roster_team_data = []
    for roster in rosters:
        roster_data = {}
        roster_data["user_id"] = roster["owner_id"]
        # roster_data["wins"] = roster["settings"]["wins"]
        # roster_data["loses"] = roster["settings"]["losses"]
        # roster_data["ties"] = roster["settings"]["ties"]
        roster_data["waiver_budget_left"] = (
            100 - roster["settings"]["waiver_budget_used"]
        )
        roster_data["division"] = roster["settings"]["division"]
        try:
            roster_data["points_potential"] = float(
                str(roster["settings"]["ppts"])
                + "."
                + str(roster["settings"]["ppts_decimal"])
            )
            roster_data["win_streak"] = roster["metadata"]["streak"]
            roster_data["record"] = roster["metadata"]["record"]
        except KeyError:
            roster_data["points_potential"] = 0
            roster_data["win_streak"] = np.nan
            roster_data["record"] = np.nan

        roster_team_data.append(roster_data)

    df = pd.DataFrame(roster_team_data)
    df["current_period"] = True

    return df

In [92]:
def sleeper_data(league_id: str, weeks: dict, year: int):
    """main entry point function"""

    # establish league object and get data of interest
    league = League(league_id)
    users = league.get_users()
    rosters = league.get_rosters()

    all_weeks = []
    for key, value in weeks.items():
        # weekly matchup data
        matchups = league.get_matchups(key)
        weekly_data = get_weekly_data(rosters, matchups, users, key, value)
        all_weeks.append(weekly_data)

    all_week_data = pd.concat(all_weeks)

    # roster data
    roster_data = get_roster_stats(rosters)

    # merging data roster data for current period only
    final_data = all_week_data.merge(
        roster_data, how="left", on=["user_id", "current_period"]
    )

    # adding missing cols
    final_data["leauge_id"] = league_id
    final_data["year"] = year

    return final_data

# TODO
1. Check why certain roster keys aren't working and add in error handling for new season
2. Add in BQ upload job
3. Build out dashboard

In [55]:
last_year_id = "860230546872176640"
current_id = "995873526081122304"

In [73]:
# week config
weeks = {
    1: True,
    2: False,
    3: False,
    4: False,
    5: False,
    6: False,
    7: False,
    8: False,
    9: False,
    10: False,
    11: False,
    12: False,
    13: False,
    14: False,
}

In [93]:
final_data = sleeper_data(last_year_id, weeks, 2022)

In [94]:
final_data

,user_id,user_name,week,opponent_id,opponent_name,points_for,points_against,points_spread,playoffs,wins,losses,ties,current_period,waiver_budget_left,division,points_potential,win_streak,record,leauge_id,year
0,460510059265257472,bhamm21,1,860346926040842240,kennedynickerson,109.06,131.92,-22.86,False,0,1,0,True,47.0,3.0,1748.10,3L,LWWWLWWWLLWLLL,860230546872176640,2022
1,860346926040842240,kennedynickerson,1,460510059265257472,bhamm21,131.92,109.06,22.86,False,1,0,0,True,47.0,3.0,1980.42,2L,WWWWLWWWWWWWLL,860230546872176640,2022
2,462745104268914688,csak,1,860235052695556096,kmpeoples,104.08,133.96,-29.88,False,0,1,0,True,87.0,3.0,1775.62,1W,LLLWWWWLWWLWLW,860230546872176640,2022
3,860235052695556096,kmpeoples,1,462745104268914688,csak,133.96,104.08,29.88,False,1,0,0,True,89.0,3.0,1688.90,2W,WLLLWLWLLWLLWW,860230546872176640,2022
4,461711675867787264,jcsalvo,1,798880355598864384,ttester9842,102.06,106.68,-4.62,False,0,1,0,True,5.0,1.0,1818.90,1L,LLLWLLLWLWLLWL,860230546872176640,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,860665827303804928,jtschwamburger,14,422091667659563008,JeanyusMusic,71.26,165.88,-94.62,False,0,1,0,False,NaN,NaN,NaN,NaN,NaN,860230546872176640,2022
164,463033097869324288,ericbmg,14,860346926040842240,kennedynickerson,150.52,82.86,67.66,False,1,0,0,False,NaN,NaN,NaN,NaN,NaN,860230546872176640,2022
165,860346926040842240,kennedynickerson,14,463033097869324288,ericbmg,82.86,150.52,-67.66,False,0,1,0,False,NaN,NaN,NaN,NaN,NaN,860230546872176640,2022
166,798880355598864384,ttester9842,14,860281658170064896,johnnywhodey,137.40,122.32,15.08,False,1,0,0,False,NaN,NaN,NaN,NaN,NaN,860230546872176640,2022


In [96]:
pivot_data = pd.pivot_table(
    final_data, values="points_for", index="week", columns="user_name"
)
pivot_data

user_name,JeanyusMusic,LeonBooker7,bhamm21,csak,ericbmg,jcsalvo,johnnywhodey,jtschwamburger,kennedynickerson,kmpeoples,mancusojoshua,ttester9842
week,,,,,,,,,,,,
1,93.42,107.40,109.06,104.08,142.52,102.06,98.40,136.72,131.92,133.96,132.40,106.68
2,117.42,120.18,123.54,128.18,172.02,100.30,72.96,143.16,134.18,97.96,97.00,98.90
3,105.20,123.46,105.24,96.60,107.92,113.28,90.96,112.00,116.76,84.58,110.08,100.64
4,140.76,89.36,105.10,130.12,117.86,138.32,79.38,92.18,136.28,135.80,96.46,128.10
5,132.76,140.94,104.14,141.36,107.16,106.86,111.56,119.48,123.50,145.62,105.08,125.18
6,115.60,100.02,96.66,125.66,114.90,104.44,116.30,82.70,124.88,92.22,82.82,92.36
7,108.82,111.34,117.02,123.48,114.90,100.18,126.00,106.24,111.56,88.92,102.12,81.42
8,150.80,95.74,149.66,132.82,150.52,152.16,114.98,94.08,145.74,125.62,79.22,111.84
9,111.22,119.60,77.82,109.20,114.32,74.96,84.10,111.24,156.30,99.38,134.54,109.08
